# Exploratory Data Analysis with Pandas


## Introduction

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# We're going to use the Python requests library which is great for accessing the web.
!pip install requests

import requests

Video tutorials: http://www.dataschool.io/easier-data-analysis-with-pandas/

A lot of the content of this notebook is based on Jake VanderPlas' excellent Python Data Science Handbook: https://jakevdp.github.io/PythonDataScienceHandbook/

## NumPy - the Foundation of Data Science in Python

Data science is largely about the efficient manipulation of collections of numbers, so to support effective data science a language needs a way to do this. In Python this is done through the NumPy libary, which are much more memory- and computation-efficient than the built-in lists in Python.

Python dicts are suboptimal for several reasons:

* they are heterogeneous
* even when storing numbers, these numbers are objects (as Python is a pure OOPL), and so are more than just (say) int32 or int64's like in C/C++; they have reference counts, type info, size info, and the actual data

Python offers an array type which is homogeneous which improves om lists as far as storage goes, but offers limited operations on that data.

NumPy bridges the gap, offering both efficient storage of homogeneous data in single or multi-dimensional arrays, and a rich set of operations on that data.

In this section we will cover some of the basics of NumPy, but our focus will be mostly on Pandas, a library built on top of NumPy that is particularly well-suited to manipulating tabular data.


In [ ]:
# Create one dimensional NumPy array from a list
a = np.array([1, 2, 3])
a

In [ ]:
# Append a value
b = a
a = np.append(a, 4)  # Note that this makes a copy; the original array is not affected
print(b)
print(a)

In [ ]:
# Get the shape and # of elements
print(np.shape(a))
print(np.size(a))

In [ ]:
# Index and slice
print(f'Second element of a is {a[1]}')
print(f'Last element of a is {a[-1]}')
print(f'Middle two elements of a are {a[1:3]}')

In [ ]:
# Create a 2-D array from a list of lists
b = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])
b

In [ ]:
# Get the shape, # of elements, and # of dimensions
print(np.shape(b))
print(np.size(b))
print(np.ndim(b))

In [ ]:
# Get the first row of b; these are equivalent
print(b[0]) 
print(b[0,:])

In [ ]:
# Get the first column of b
print(b[:,0])

In [ ]:
# Get a subsection of b, from 1,1 through 2,2 (i.e. before 3,3)
print(b[1:3,1:3])

In [ ]:
# Create an array of zeros of length n
np.zeros(5)

In [ ]:
# Create an array of 1s
np.ones(5)

In [ ]:
# Creat an array of 10 random integers between 1 and 100
np.random.randint(1,100, 10)

In [ ]:
# Create linearly spaced array of 5 values from 0 to 100
np.linspace(0, 100, 5)

### UFuncs

NumPy supports highly efficient operations on arrays called UFuncs (Universal Functions).

In [ ]:
np.mean(b)  # Get the mean of all the elements

In [ ]:
np.power(b, 2)  # Raise every element to second power

You can get the details on UFuncs here: https://docs.scipy.org/doc/numpy-1.13.0/reference/ufuncs.html

## Pandas Series and Dataframes

A Pandas Series is a one-dimensional array of indexed data. It wraps a sequence of values and a sequence of indices. The values are a NumPy array, while the indices are an instance of a pd.Index object.


In [ ]:
data = pd.Series([1, 4, 9, 16, 25])
data

In [ ]:
data.values

In [ ]:
data.index

Normal indexing and slicing operations are available:

In [ ]:
data[2]

In [ ]:
data[2:4]

Where NumPy arrays have implicit integer sequence indices, Pandas indices are explicit and need not be integers:

In [ ]:
data = pd.Series([1, 4, 9, 16, 25], index=['square of 1', 'square of 2', 'square of 3', 'square of 4', 'square of 5'])
data

In [ ]:
data['square of 3']

As you can see, a Series is a lot like a Python dict (with additional slicing), and we can construct one from a Python dict:

In [ ]:
pd.Series({'square of 1':1, 'square of 2':4, 'square of 3':9, 'square of 4':16, 'square of 5':25})

A number of dict-style operations work on a Series:

In [ ]:
'square of 5' in data

In [ ]:
data.keys()

In [ ]:
data.items()  # Iterable

In [ ]:
list(data.items())

In [ ]:
data.values()  # Unlike Python dict, this is not the same

In [ ]:
data['square of 6'] = 36  # We can add new entries
data

In [ ]:
data['square of 6'] = -1  # And change existing values
data

In [ ]:
del data['square of 6']  # And delete a value
data

Series also behaves a lot like a list. We saw some indexing and slicing earlier. This can be done on non-numeric indexes too:

In [ ]:
data['square of 2': 'square of 4']  # This can be confusing as it INCLUDES the final value

In [ ]:
data['square of 2': 'cube of 4']  # Be aware - a missing key will result in empty results

A DataFrame is like a dictionary that maps column names to Series objects that share the same index.

In [ ]:
names = pd.Series(['Alice', 'Bob', 'Carol'])
phones = pd.Series(['555-123-4567', '555-987-6543', '555-245-6789'])

df = pd.DataFrame({'Name': names, 'Phone': phones})
df

In [ ]:
df.index  # Like Series, DataFrame has an index for rows

In [ ]:
df.columns  # DataFrame also has an index for columns

In [ ]:
df.values

In [ ]:
df['Name']  # Acts similar to dictionary; returns Series

In [ ]:
df.Name  # You can also access columns like this. Occasionally breaks if there is a name conflcit, like 'count'

In [ ]:
# You can add new columns. Later we'll see how to do this as a function of existing columns
df['Closed'] = True
df.head()

There are many ways to construct a DataFrame. For example, from a Series or dictionary of Series, from a list of Python dicts, or from a 2-D NumPy array. There are also utility functions to read data from disk into a DataFrame, e.g. from a .csv file or an Excel spreadsheet. We'll cover some of these later.

Many DataFrame operations take an `axis` argument which defaults to zero. This specifies whether we want to apply the operation by rows (axis=0) or by columns (axis=1).

## Indexes

The Pandas Index can be thought of as an immutable ordered multiset (multiset as indices need not be unique). The immutability makes it safe to share an index between multiple columns of a DataFrame. The set-like properties are useful for things like joins (a join is like an intersection between Indexes). Let's look at some example operations to get more familiar with how they work:

In [ ]:
# Let's create two Indexes for experimentation

i1 = pd.Index([1, 3, 5, 7, 9])
i2 = pd.Index([2, 3, 5, 7, 11])

In [ ]:
i1[2]  # We can index like an array with []

In [ ]:
i1[2:5]  # And slice

In [ ]:
i1 & i2  # Intersection

In [ ]:
i1 | i2  # Union

In [ ]:
i1 ^ i2  # Difference

Series and DataFrames have an explicit Index but they also have an implicit index like a list. If the Index uses integer values things can get confusing. In such cases it is good to be explicit; there are attributes for this:

- `.loc` references the explicit Index
- `.iloc` references the implicit Index

The Python way is "explicit is better than implicit" so when indexing/slicing it is better to use these. The example below illustrates the difference:

In [ ]:
# Note: explicit index starts at 1; implicit index starts at 0
s = pd.Series(['first', 'second', 'third', 'fourth'], index=[1, 2, 3, 4]) 

print(f'Item at explicit index 1 is {s.loc[1]}')
print(f'Item at implicit index 1 is {s.iloc[1]}')
print(s.loc[1:3])
print(s.iloc[1:3])

## Loading a CSV into a Dataframe

Use `Pandas.read_csv` to read a CSV file into a dataframe. There are many optional argumemts that you can provide, for example to set or override column headers, skip initial rows, treat first row as containing column headers, specify the type of columns (Pandas will try to infer these otherwise), skip columns, and so on. The `parse_dates` argument is especially useful for specifying which columns have date fields as Pandas doesn't infer these.

Full docs are at https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

In [ ]:
df = pd.read_csv('http://samplecsvs.s3.amazonaws.com/SacramentocrimeJanuary2006.csv',
                 parse_dates=['cdatetime'])
df.head()

## Loading a Spreadsheet into a DataFrame

Use Pandas.read_excel to load spreadsheet data. Full details here: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_excel.html

In [ ]:
df = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls')
df.head()

## Saving a Dataframe to a CSV or Excel spreadsheet

You can use DataFrame.to_csv to write a DataFrame to a csv, and DataFrame.to_excel to save as a spreadsheet.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_excel.html

## Filtering

In [ ]:
import seaborn as sns;

# Get some sample data
titanic = sns.load_dataset('titanic')
titanic.head()

In [ ]:
# A Boolean expression on a Series will return a Series of Booleans
titanic.survived == 1

In [ ]:
# If you index a Series with a Boolean Series, you will select the items where the index is True.
# So:
titanic[titanic.survived == 1]

In [ ]:
# You can combine these with & and | for and and or
# Pandas must use these normally bitwise operators because Python allows them to be overloaded
# while 'and' and 'or' cannot be.
# Unfortunately as these have higher operator precedence than relational operators, the 
# subexpressions we use with them need to be enclosed in parentheses.

titanic[titanic.survived & (titanic.sex == 'female') & (titanic.age > 50)]

## Concatenation

`pandas.concat` can be used to concatenate Series and DataFrames:

In [ ]:
s1 = pd.Series(['A', 'B', 'C'])
s2 = pd.Series(['D', 'E', 'F'])
pd.concat([s1, s2])

Note that the Indexes are concatenated too. If you are using a simple row number index and don't want this behavior use the `ignore_index` argument:

In [ ]:
pd.concat([s1, s2], ignore_index=True)

Alternatively you can use `verify_integrity=True` to cause an exception to be raided if the result would have duplicate indices.

In [ ]:
pd.concat([s1, s2], verify_integrity=True)

In [ ]:
d1 = pd.DataFrame([['A1', 'B1'],['A2', 'B2']], columns=['A', 'B'])
d2 = pd.DataFrame([['C3', 'D3'],['C4', 'D4']], columns=['A', 'B'])
d3 = pd.DataFrame([['B1', 'C1'],['B2', 'C2']], columns=['B', 'C'])
pd.concat([d1, d2])

In [ ]:
# We can join on other axis too.
pd.concat([d1, d2], axis=1)

In [ ]:
pd.concat([d1, d3], axis=1)

In [ ]:
# If the columns are not completely shared, additional NaN entries will be made.
pd.concat([d1, d3])

In [ ]:
# We can force concat to only include the columns that are shared with an inner join.
pd.concat([d1, d3], join='inner')

See https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html for more options.

## Merging and Joining

Pandas has a `merge` function that supports one-to-one, many-to-one and many-to-many joins. merge will look for matching column names between the inputs and use this as the key:

In [ ]:
d1 = pd.DataFrame({'city': ['Seattle', 'Boston', 'New York'], 'population': [704352, 673184, 8537673]})
d2 = pd.DataFrame({'city': ['Boston', 'New York', 'Seattle'], 'area': [48.42, 468.48, 142.5]})
pd.merge(d1, d2)

In [ ]:
# You can explicitly specify the column to join on; this is equivalent to the above example:
pd.merge(d1, d2, on='city')

In [ ]:
# If the column names don't match you can specify the names to use:
d3 = pd.DataFrame({'place': ['Boston', 'New York', 'Seattle'], 'area': [48.42, 468.48, 142.5]})
pd.merge(d1, d3, left_on='city', right_on='place')

In [ ]:
# If you want to drop the redundant column:
pd.merge(d1, d3, left_on='city', right_on='place').drop('place', axis=1)

`merge` joins on arbitrary columns; if you want to join on the index you can use `left_index` and `right_index`:

In [ ]:
df1 = pd.DataFrame(list('ABC'), columns=['c1'])
df2 = pd.DataFrame(list('DEF'), columns=['c2'])
pd.merge(df1, df2, left_index=True, right_index=True)

Pandas provides a utility method on DataFrame, `join`, to do the above:

In [ ]:
df1.join(df2)

`merge` can take a `how` argument that can be `inner`, `outer`, `left` or `right` to control the type of join. `inner` joins are the default.

For more info on merging see https://pandas.pydata.org/pandas-docs/stable/merging.html

## Aggregating

In [ ]:
!pip install seaborn

In [ ]:
import seaborn as sns;

titanic = sns.load_dataset('titanic')
titanic.head()

In [ ]:
# Use unique() to see the full set of distinct values in a series
titanic.deck.unique()

In [ ]:
# describe() will give summary statistics on a DataFrame. We first drop rows with NAs.
titanic.dropna().describe()

In [ ]:
titanic.groupby('sex')['survived'].mean()

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].mean()

The DataFrame result is an example of a multi-indexed DataFrame (indexed by both 'sex' and 'class'). We're mostly going to ignore those in this notebook, but it is worth noting that Pandas has an `unstack` method that can turn a mutiply-indexed DataFrame back into a conventionally-indexed one:

In [ ]:
titanic.groupby(['sex', 'class'])['survived'].mean().unstack()

In [ ]:
# All of the above can be achieved with a convenience pivot table method
titanic.pivot_table('survived', index='sex', columns='class')

In [ ]:
# Let's break things down further by age group
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', index=['sex', age], columns='class')

In [ ]:
# Index and colummns are also the second and third positional arguments, so we could just use:
titanic.pivot_table('survived', ['sex', age], 'class')

## Applying Functions

We saw earlier that we can add new columns to a DataFrame easily. The new column can be a function of an existing column. For example, we could add an 'is_adult' field to the Titanic data:

In [ ]:
titanic['is_adult'] = titanic.age >= 18
titanic.head()

That's a simple case; we can do more complex row-by-row applications of arbitrary functions; here's the same change done differently (this would be much less efficient but may be the only option if the function is complex):

In [ ]:
titanic['is_adult'] = titanic.apply(lambda row: row['age'] >= 18, axis=1)
titanic.head()

## String Operations

Pandas has vectorized string operations that will skip over missing values. You can read about them here; we wil show a few examples: https://pandas.pydata.org/pandas-docs/stable/text.html

In [ ]:
# Let's get the more detailed Titanic data set
df = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls')
df.head()

In [ ]:
# Upper-case the home.dest field
df['home.dest'].str.upper().head()

In [ ]:
# Let's split the field up into two
place_df = df['home.dest'].str.split('/', expand=True)  # Expands the split list into DF columns
place_df.columns = ['home', 'dest', '']  # For some reason there is a third column
df['home'] = place_df['home']
df['dest'] = place_df['dest']
df = df.drop(['home.dest'], axis=1)
df

## Handling Missing Data

To see if there are missing values, we can use isnull() to get a DataFrame showing the rows that have nulls, and where they have them:

In [ ]:
df.isnull().head()

The above will show us the first few rows that had null values. If we want to know which columns may have nulls, we can use:

In [ ]:
df.isnull().any()

To drop rows that have missing values, use dropna(); add `inplace=True` to do it in place.

In [ ]:
df.dropna().head()

In this case there are none - no-one could both be on a boat and be a recovered body, so at least one of these fields is always NaN.

## Charting

See the Python Graph Gallery at https://python-graph-gallery.com/ for many examples of different types of charts including the code used to create them.

There are many plotting libraries for Python; the most well known are matplotlib, seaborn (which extends matplotlib), Bokeh, and Plotly. Some offer more interactivity than others. We will use Seaborn for our examples. We first need to use the following magic to get the plots to show up in Jupyter:

In [ ]:
%matplotlib inline

In [ ]:
# Let's get the more detailed Titanic data set
df = pd.read_excel('http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls')
df.head()

In [ ]:
# We can use a factorplot to count categorical data
import seaborn as sns
sns.factorplot('sex',data=df,kind='count')

In [ ]:
# Let's bring class in too:
sns.factorplot('pclass', data=df, hue='sex', kind='count')

In [ ]:
# Of course we can aggregate the other way too
sns.factorplot('sex', data=df, hue='pclass', kind='count')

In [ ]:
# Let's see how many people were on each deck
deck = pd.DataFrame(df['cabin'].dropna().str[0])
deck.columns = ['deck']
sns.factorplot('deck', data=deck, kind='count')

In [ ]:
# What class passenger was on each deck?
df2 = df[['cabin', 'pclass']]
df2 = df2.dropna()
df2['deck'] = df2.apply(lambda row: ord(row.cabin[0]) -64, axis=1)

sns.regplot(x=df2["pclass"], y=df2["deck"])

## Summarizing Data with pandas_profiling and facets

In [ ]:
!pip install pandas-profiling

In [ ]:
import pandas_profiling
import seaborn as sns;

titanic = sns.load_dataset('titanic')

pandas_profiling.ProfileReport(titanic)  # You may need to run cell twice

Facets is a new library from Google that looks very good. It has similr functionality to pandas_profiling as well as some powerful visualization. Installation is more complex so we won't use it now but it is worth considering.

https://github.com/pair-code/facets


## Sample: Loading JSON into a DataFrame and Expanding Complex Fields

In this example we'll see how we can load som structured data and process it into a flat table form better suited to machine learning.

In [ ]:
# Let's get some data; top stories from lobste.rs; populate a DataFrame with the JSON
stories = pd.read_json('https://lobste.rs/hottest.json')
stories.head()

In [ ]:
# Use the "short_id' field as the index
stories = stories.set_index('short_id')

# Show the first few rows
stories.head()

In [ ]:
# Take a look at the submitter_user field; it is a dictionary itself.
stories.submitter_user[0]

In [ ]:
# We want to expand these fields into our dataframe. First expand into its own dataframe.
user_df = stories.submitter_user.apply(pd.Series)
user_df.head()

In [ ]:
# We should make sure there are no collisions in column names.
set(user_df.columns).intersection(stories.columns)

In [ ]:
# We can rename the column to avoid the clash
user_df = user_df.rename(columns={'created_at': 'user_created_at'})

In [ ]:
# Now combine them, dropping the original compound column that we are expanding.
stories = pd.concat([stories.drop(['submitter_user'], axis=1), user_df], axis=1)
stories.head()

In [ ]:
# The tags field is another compound field.
stories.tags.head()

In [ ]:
# Make a new dataframe with the tag lists expanded into columns of Series.
tag_df = stories.tags.apply(pd.Series)
tag_df.head()

In [ ]:
# Pivot the DataFrame
tag_df = tag_df.stack()
tag_df

In [ ]:
# Expand into a 1-hot encoding
tag_df = pd.get_dummies(tag_df)
tag_df

In [ ]:
# Merge multiple rows
tag_df = tag_df.sum(level=0)
tag_df

In [ ]:
# And add back to the original dataframe
stories = pd.concat([stories.drop('tags', axis=1), tag_df], axis=1)
stories.head()

## Sample 2: Baby Names

In this sample some of the tasks will be left as exercises for you. The data comes from US census and is the count of names of children born in years from 1880 to 2014.

In [ ]:
df = pd.read_csv('NationalNames.csv.zip', compression='zip')  # Pandas can unzip the data for you
df.head()

In [ ]:
# Exercise: show the baby names from 1918

In [ ]:
# Exercise: get the counts per year for the name 'John'

In [ ]:
# Exercise: do the same but restrict to boys now!

In [ ]:
# Exercise: plot popularity of John per year (hint: look at help for Seaborn barplots)